# CSCI-UA 0473 - Introduction to Machine Learning
## Homework 4
### Assignment 4 - Adaptive RBFN and Stochastic Gradient Descent (25 Points)

The task for this homework is again that of handwritten digit recognition on the MNIST dataset.  **For this time, you will train on the full dataset (that is, 60000 training images plus 10000 test images), rather than a small subset.**

Your first goal is to implement an adaptive radial basis function network, and train it with stochastic gradient descent (SGD), while going over a few questions.

<div class="alert alert-danger">
**Please do not miss any text with red background; they are either questions you need to answer, or directions to implement some functionality.**
</div>

<div class="alert alert-warning">
**Please start this homework early.**  Stochastic gradient descent will take a particularly long time to train.
</div>

<div class="alert alert-warning">
**Please submit a fully-run notebook; not doing so would incur at least 10% penalty problem-wise.**
</div>

In [ ]:
import autograd.numpy as np
import autograd.scipy as ascipy
import autograd.misc.flatten
from autograd import value_and_grad
import scipy.optimize
import pickle

import matplotlib.pyplot as plot
%matplotlib inline

## 1. Data

In [ ]:
npz = np.load('mnist.npz')
# Labels
train_labels = npz['train_labels']
test_labels = npz['test_labels']
# Data.  We linearly scale the integers into something between 0 and 1.
train_data = npz['train_data'] / 255.
test_data = npz['test_data'] / 255.

The shapes and types for each numpy array is:

* `train_data`: 60000x28x28 float64 array
* `train_labels`: 60000 int64 array
* `test_data`: 10000x28x28 float64 array
* `test_labels`: 10000 int64 array

The digit for i-th training data (`train_data[i]`) is `train_labels[i]`.  The same holds for test data.

In [ ]:
plot.matshow(train_data[0], vmin=0, vmax=1, cmap='gray')
# See help(str.format) or https://docs.python.org/3.1/library/string.html#formatspec for usage of the Pythonic formatter
# If you are comfortable with C you can also write things like sprintf(): 'Digit %d' % (train_labels[0])
plot.title('Digit {}'.format(train_labels[0]))

## 2. Model Definition

The first thing we need is model definition.  Recall that the Gaussian Radial Basis Function we have learned in the class transforms an arbitrary input vector $\mathbf{x} \in \mathbb{R}^d$ into another vector $\phi(\mathbf{x}) \in \mathbb{R}^k$ based on a set of bases $\mathbf{r}_k$:

$$
\phi(\mathbf{x})_k = \exp\left(\lVert \mathbf{x} - \mathbf{r}_k \rVert^2\right) = \exp \left( \sum_{i=1}^d (x_i - r_{ki})^2\right)
$$

An alternative would be taking an average of the squares instead of summing them up:

$$
\phi(\mathbf{x})_k = \exp\left(\lVert \mathbf{x} - \mathbf{r}_k \rVert^2\right) = \exp \left( \dfrac{1}{d} \sum_{i=1}^d (x_i - r_{ki})^2\right)
$$

<div class="alert alert-danger">
(5 points) Between the two options, which one do you think that would work better?  Justify your answer and replace the `XXX` in the `adaptive_rbfn_logits` function with your choice.  **Note: "by experiment" is not a sufficient justification for *this* problem.**
</div>

**Double click this sentence**, replace it with *your answer and/or LaTeX equations* (like $a=b+c$) here, then press `Ctrl+Enter`.

In [ ]:
def adaptive_rbfn_logits(x, params):
    '''
    x: numpy array of training data, with shape (n_samples, 28, 28)
    params: a dictionary with "bases", "w" and "b" as keys.
    '''
    bases = params['bases']
    w = params['w']
    b = params['b']
    x = x.reshape((x.shape[0], -1))
    
    phi = np.exp(-((x[:, None, :] - bases[None, :, :]) ** 2).XXX(axis=-1)) # (n_samples, n_bases)
    logits = np.dot(phi, w) + b
    
    return logits

The function above applies a linear transformation on $\phi(\mathbf{x})$ to get the *logits* for each class.  The logits can be considered as "scores"; the higher the logit, the more likely the class.  In fact, the probability of a certain class can be directly computed by applying a *softmax* on the logits.

Recall that the multiclass logistic regression minimizes the negative log-likelihood (sometimes also called *cross entropy*) of the entire dataset:

$$
J(M, D_\text{tra}) = -\dfrac{1}{N} \sum_{(\mathbf{x}, y^*) \in D_\text{tra}}
\log \dfrac{\exp \mathbf{w}^T_{y^*} \tilde{\mathbf{x}}}{\sum_y \exp \mathbf{w}^T_y \tilde{\mathbf{x}}}
$$

<div class="alert alert-danger">
(5 points) The following function implements the cost function in a naive, numerically unstable way.  Please rewrite the indicated block **in at most 2 lines** so that it becomes numerically stable.  You will only get 2 points if your replacement is correct and more than 2 lines.
</div>

<div class="alert alert-info">
**Hint:** Recall Problem 2 in Homework 3, then look at `autograd.scipy.misc`, then see how we select elements efficiently (which is usually called [fancy](https://docs.scipy.org/doc/numpy/user/basics.indexing.html#index-arrays) [indexing](https://docs.scipy.org/doc/numpy/user/basics.indexing.html#indexing-multi-dimensional-arrays)). <br/>

Other autodifferentiation frameworks also have their numerically stable log-softmax functions.
</div>

In [ ]:
def adaptive_rbfn_cost(param_buf, x, y, unpack):
    '''
    The cost function for ARBFN.
    x: numpy array of training data, with shape (n_samples, 28, 28)
    y: numpy array of training labels, with shape (n_samples,).  All elements are integers.
    param_buf: the *flattened* numpy array containing all the parameters.
    unpack: a function that recovers the flattened parameters into the original list/dict.
    
    param_buf and unpack are quite a nuisance as they are introduced by autograd.
    A more detailed explanation comes with the "autograd.misc.flatten" function below.
    '''
    params = unpack(param_buf)
    logits = adaptive_rbfn_logits(x, params)
    
    #### BEGINNING OF CODE TO BE REPLACED
    exp_logits = np.exp(logits)
    softmax = exp_logits / exp_logits.sum(axis=1, keepdims=True)
    log_probs = np.log(softmax[np.arange(x.shape[0]), y])
    cost = -log_probs.mean()
    #### END OF CODE TO BE REPLACED
    
    return cost

In [ ]:
# This statement returns a function that has the same signature as adaptive_rbfn_cost(),
# but returns both the cost and the gradient.
adaptive_rbfn_cost_and_grad = value_and_grad(adaptive_rbfn_cost)

In [ ]:
def train_full_batch(cost_and_grad, initial_params, train_data, train_labels, use_scipy=True, learning_rate=1, max_iter=30000):
    # NOTE: We should have early stopping here, but it is left for you to implement in the next problem (not here).
    # Autograd.misc.flatten() is a tricky function that allows us to compute the gradients to all parameters
    # with a single grad() call.
    # It takes in a list or dictionary, and returns two objects:
    # (1) a flattened, i.e. 1D, numpy array which contains all the parameters, and
    # (2) an unflatten function, which will recover the original list/dictionary structure from the flattened array.
    # Collapsing all the parameters into a single array would also allow us to use scipy's optimize function which
    # only takes one input variable.
    param_buf, unpack = autograd.misc.flatten(initial_params)

    if not use_scipy:
        for i in range(max_iter):
            loss, dparam_buf = cost_and_grad(param_buf, train_data, train_labels, unpack)
            param_buf -= learning_rate * dparam_buf
            if i % 100 == 0:
                print(i, loss)
    else:
        model = scipy.optimize.minimize(cost_and_grad, param_buf, (train_data, train_labels, unpack), jac=True, method='CG')
        print(model)
        param_buf = model.x
    
    return unpack(param_buf)

Now that we have defined a function for training an adaptive RBFN, let's see how a small model works on a small dataset (100 training samples, 10 test samples).

In [ ]:
train_size = 100
test_size = 10
n_bases = 10
n_features = np.prod(train_data.shape[1:])

params = {}
params['bases'] = np.random.rand(n_bases, n_features)
params['w'] = np.random.randn(n_bases, 10) * 0.1
params['b'] = np.zeros(10)

In [ ]:
%%time
new_params = train_full_batch(adaptive_rbfn_cost_and_grad, params, train_data[:train_size], train_labels[:train_size], False)

In [ ]:
y_hat = adaptive_rbfn_logits(test_data[:test_size], new_params).argmax(axis=1)
print(np.equal(y_hat, test_labels[:test_size]).sum(), '/', test_size)

When the gradient is computed from the entire training set, we usually call it *(full) batch gradient descent*.  However, batch gradient descent is time- and memory-consuming, in the sense that it has to go through all the training examples and average all the gradients for a *single* step, and it quickly becomes impractical if the dataset or the model becomes large.

The solution is *not* to see all the examples for a single gradient step.  Instead, for each step, we compute and average the gradients for only a small subset (commonly called a *minibatch*) of examples.  This is called *minibatch gradient descent*, or *stochastic gradient descent* (SGD), though the latter refer to the special case where the minibatch only contains a single example in some literatures.

The intuition behind SGD is that the minibatch gradient is a noisy but unbiased estimate of the true gradient.  This of course has pros and cons.

The main downside is that, due to the inherent noise of minibatch gradient, SGD no longer guarantees to converge to a local minimum, and sometimes it will fluctuates or even overshoot itself.  You will have to use a significantly smaller, and sometimes decaying, learning rate.  Very often, you would see a curve of training loss like [this](http://cs231n.github.io/neural-networks-3/#loss):

![](http://cs231n.github.io/assets/nn3/loss.jpeg)

The obvious upside is that estimating the gradient using minibatch is much more efficient than computing the true gradient across all training examples.  Or, after we iterate over the entire training set, we had already performed a lot of gradient steps, which is not necessarily a bad thing.  In fact, the size of a minibatch itself can be considered as a hyperparameter to fiddle with ~~although in practice few people do that~~.

In practice, SGD often works good enough, especially for large models (such as neural networks) on large datasets.

See [Leon Bottou's SGD Tricks](https://cilvr.cs.nyu.edu/diglib/lsml/bottou-sgd-tricks-2012.pdf) if you are interested in the details of SGD.

<div class="alert alert-danger">
(15 points) Implement SGD, and train your ARBFN model with 20 bases on the whole dataset, with the size of minibatch being 10 and a maximum of 2000 epochs.  <br/>
Plot the training loss and validation accuracy, and report the test accuracy. <br/>
Do the same for minibatch size 100 and answer this question in the cell below: *What do you observe from the learning curve and training time of different minibatch sizes?*  <br/>

You are free to define your own `train_sgd` function.  **Please do not use `autograd.misc.optimizers` package.**
</div>

<div class="alert alert-info">
**Hint:** To save your time figuring out the complete workflow of SGD, here is how we usually do it.
<ol>
    <li> Split the training set into a validation set and another smaller training set. </li>
    <li> Shuffle the (rest of the) training set.  Of course, you can shuffle the indices rather than the actual items. </li>
    <li> Divide the training set into equally-sized minibatches.  The last batch may not have the same size as the others, and it's up to you how to deal with it (usually it doesn't matter too much). </li>
    <li> Iterate over each minibatch, compute the gradient, and update the parameters. </li>
    <li> After going through all the minibatches (we call that *an epoch*), evaluate the model on validation set, and decide whether to early stop.  Since SGD will fluctuate a lot, you may want to leave it run for a little while longer even if the validation error does not seem to decrease.  </li>
    <li> Repeat 2-5 until done. </li>
</ol>

Optionally, you can also decay the learning rate after each minibatch or each epoch.
</div>

<div class="alert alert-warning">
**Warning:** This assignment would take potentially hours to train a model to "good performance" on an average laptop.  In practice, it is very common for neural networks to take hours, days or even weeks to train, even if powered by GPUs which are good at matrix multiplication and elementwise computation.
</div>

**Double click this sentence**, replace it with *your answer and/or LaTeX equations* (like $a=b+c$) here, then press `Ctrl+Enter`.

In [ ]:
# YOUR CODE HERE
def train_sgd():
    pass

For reference, here is my training loss and validation accuracy curve running for 2000 epochs.

![](curve.png)

### Assignment 5: k-NN Classifier (15 points)

The second goal is to compare you ARBFN against a k-NN classifier.  For the k-NN classifier, you need to try a couple $k$'s and pick the best one in a programmatic way.

<div class="alert alert-danger">
(15 points) Build a k-NN classifier using `scikit-learn` with $k$'s from 1 to 3, train it on the first 1000 examples of MNIST dataset, and report the accuracy on the full test set. <br/>

Would you use the (exact) k-NN classifier you have built for very large dataset (with maybe millions of examples)?  Explain your answer.
</div>

In [ ]:
# YOUR CODE HERE

**Double click this sentence**, replace it with *your answer and/or LaTeX equations* (like $a=b+c$) here, then press `Ctrl+Enter`.